In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

pd.options.display.max_columns = None
#pd.set_printoptions(max_columns=500)

TRAIN = pd.read_csv("../datasources/mnsit/train.csv", delimiter=',') #, skiprows=1)
TEST = pd.read_csv("../datasources/mnsit/test.csv", delimiter=',') #, skiprows=1)
X_TRAIN = TRAIN.copy()
X_TEST = TEST.copy()
y = TRAIN.label
del X_TRAIN["label"]

# Fonction d'export pour kaggle
def ExportKaggle(algo):
    p_test = algo.predict(X_TEST)
    result = pd.DataFrame(X_TEST.index + 1, columns=['ImageId'])
    pred = pd.DataFrame(p_test, columns=['Label'])
    result = result.join(pred)
    fichier = "./data/result_" + str(datetime.datetime.now()).replace(":", "-").replace(" ", "_")
    result.to_csv(fichier, columns=["ImageId", "Label"], index=False)

# Essayons maintenant  avec un autre classifier (RandomForest)

In [2]:
#kn = KNeighborsClassifier(n_neighbors=2)
#kn.fit(X_TRAIN, y)
rf = RandomForestClassifier(n_estimators=1000, random_state=3, max_features=1)
rf.fit(X_TRAIN, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=1, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=3, verbose=0, warm_start=False)

In [3]:
print ("Score Train -->", round(rf.score(X_TRAIN, y) *100,2), " %")

Score Train --> 100.0  %


C'est bien mieux que la DGS (à 85.88%). Néanmoins il est évident que le modèle est en overfit ! il va falloir ajouter des données afin de l'ajuster mieux.

In [4]:
ExportKaggle(rf)

### Score Kaggle 0.96071 (RF / n_estimators=1000, random_state=3, max_features=1)

## Recherche des meilleurs hyper-parametres

In [ ]:
param_grid_rf = { 'n_estimators' : [800, 1000], 
               'max_features' : [1, 0.5, 0.2],
               'random_state' : [3, 4, 5]}
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5)
grid_search_rf.fit(X_TRAIN, y)
print ("Score final : ", round(grid_search_rf.score(Xtrain, y) *100,4), " %")
print ("Meilleurs parametres: ", grid_search_rf.best_params_)
print ("Meilleure config: ", grid_search_rf.best_estimator_)

In [ ]:
ExportKaggle(param_grid_rf)